In [1]:
#!pip install gym
#!pip install tensorflow[and-cuda]
#!pip install pygame

In [1]:
# set tf warning level to 2 ....shows errors but not warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#tf.__version__

In [1]:
from gym import Env
from gym.spaces import Discrete, Box

import pandas as pd


import tensorflow as tf
#tf.config.run_functions_eagerly(True)
#print(tf.config.functions_run_eagerly())
from tensorflow import keras
from tensorflow.keras.models import load_model


#from rl.agents import DQNAgent
#from rl.policy import BoltzmannQPolicy
#from rl.memory import SequentialMemory


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# tyr legacy adam due to numpy error
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers.legacy import Adam

import numpy as np
import random

import pygame

2024-09-24 14:32:14.494756: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-24 14:32:14.503154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 14:32:14.513224: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 14:32:14.516236: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-24 14:32:14.524164: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Check if TensorFlow is built with CUDA support
if tf.config.list_physical_devices('GPU'):
    print("CUDA is available")
    print(f"Device: {tf.config.list_physical_devices('GPU')[0]}")
else:
    print("CUDA is not available")

CUDA is available
Device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


I0000 00:00:1727184738.599776   15305 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727184738.631297   15305 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727184738.632836   15305 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


# Create Environment

In [3]:
# ubuntu
df_normalised = pd.read_csv("/home/loz/Documents/GitHub/MSc-Project/SWaT_12_23_sec_minmax.csv", index_col='time_in_seconds')
data_array = df_normalised.to_numpy()
#df_normalised.shape

In [4]:
#data_array

In [5]:
component_pos = {key: idx for idx, key in enumerate(df_normalised.columns)}
component_pos.items()

dict_items([('FIT101', 0), ('LIT101', 1), ('MV101', 2), ('P101', 3), ('AIT201', 4), ('AIT202', 5), ('AIT203', 6), ('FIT201', 7), ('MV201', 8), ('P203', 9), ('P205', 10), ('DPIT301', 11), ('FIT301', 12), ('LIT301', 13), ('MV301', 14), ('MV302', 15), ('MV303', 16), ('MV304', 17), ('P301', 18), ('P302', 19), ('AIT401', 20), ('AIT402', 21), ('FIT401', 22), ('LIT401', 23), ('AIT501', 24), ('AIT502', 25), ('AIT503', 26), ('AIT504', 27), ('FIT501', 28), ('FIT502', 29), ('FIT503', 30), ('FIT504', 31), ('PIT501', 32), ('PIT502', 33), ('PIT503', 34), ('FIT601', 35), ('P602', 36)])

In [6]:
# test lstm swat simulator
# Load the model from the SavedModel directory
# 20 secs
#swat_lstm = tf.keras.models.load_model('swat_lstm_1_0.keras')
#swat_lstm = tf.keras.models.load_model('best_model.keras')
#model_window = 20
swat_lstm = tf.keras.models.load_model('swat_lstm_100s.keras')
model_window = 100

I0000 00:00:1727184740.103648   15305 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727184740.105121   15305 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727184740.106435   15305 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727184740.191790   15305 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [7]:
# Reshape the input from data_array to add the batch dimension
reshaped_input = data_array[:model_window-1, :].reshape(1, model_window-1, data_array.shape[1])  # (1, 99, 37)
# round to make observation space workable
reshaped_input = np.round(reshaped_input, 2)

In [8]:

#reshaped_input.shape
#reshaped_input[0,-1,]

# get last row
one_state = reshaped_input[0,-1,:]
one_state.shape

In [ ]:
for _ in range(10):
    prediction = swat_lstm.predict(reshaped_input)
    # Print shape
    #print(prediction.shape)
    # drop first row from input, add prediction with new axis padded to match shape
    reshaped_input = np.concatenate((reshaped_input[:, 1:, :], prediction[:, np.newaxis, :]), axis=1)
    #print(reshaped_input.shape)


In [59]:
prediction = swat_lstm.predict(reshaped_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [60]:
def update_state(state_history, action_input):
    # add action to previous states
    model_input = np.concatenate((state_history[:, 1:, :], action_input[:, np.newaxis, :]), axis=1)
    # predict
    new_state = swat_lstm.predict(model_input)
    return new_state, model_input
    


In [61]:
new_state, model_input = update_state(reshaped_input, prediction) 
new_state.shape

#new_state[0]=10
#new_state

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


(1, 37)

In [9]:
#model_input.shape

In [10]:
#reshaped_input.shape

In [11]:
component_pos['MV201']

8

In [62]:
# inherit from gym env
class SwatEnv(Env):
    def __init__(self, model_input):
        print('init reached')
        # 0: both off, 1: MV101 On MV201 Off, 2: MV101 Off MV201 On, 3: both on
        self.action_space = Discrete(4)
        
        # swat value array, only position 1 used as lit101 level. 2.d.p 
        self.observation_space = Box(
            # round to 2 d.p. to act as binning values
            low=np.round(np.full((1, 37), 0.00), 2),  # 1x37 array with all elements set to 0.00
            high=np.round(np.full((1, 37), 100.00), 2),  # 1x37 array with all elements set to 100.00
            dtype=np.float32  # Specify float with two decimal places precision
        )
        
        # state history is 98 time steps preceding
        self.state_history = model_input
        # set start state as last row in modelinput
        self.state = self.state_history [0,-1,:]
        # set shower length
        self.episode_length = 60
    
    # Override inherited method  
    def sample_observation(self):
        return np.round(self.observation_space.sample(), 2)

        
    def update_state(self, state_history, action_input):
        # Ensure action_input is reshaped to 1x37 array
        action_input = action_input.reshape(1, -1)
        
        # add action to previous states
        model_input = np.concatenate((state_history[:, 1:, :], action_input[:, np.newaxis, :]), axis=1)
        # predict
        new_state = swat_lstm.predict(model_input)
        
        # update history with new state
        self.state_history = np.concatenate((state_history[:, 1:, :], new_state[:, np.newaxis, :]), axis=1)
        
        # flatten new_state from (1,37) to (37,)
        new_state = new_state.flatten()
        
        return new_state
        
    def step(self, action):
        #print(f'step reached {action}')
        # apply action
        # subtract 1 fro action so 1 - 1 = 0 no change
        #self.state = self.state + action -1
        
        # copy current state
        action_input = self.state.copy() 
        
        # MV101 
        if action == 0:
            ## MV101 and MV201 off         
            action_input[component_pos['MV101']] = 0
            action_input[component_pos['MV201']] = 0
            
        elif action == 1:
            # MV101 On, MV201 Off
            action_input[component_pos['MV101']] = 1
            action_input[component_pos['MV201']] = 0
        elif action == 2:
            # MV101 Off, MV201 On
            action_input[component_pos['MV101']] = 0
            action_input[component_pos['MV201']] = 1
        elif action == 3:
            # Both MV101 and MV201 On
            action_input[component_pos['MV101']] = 1
            action_input[component_pos['MV201']] = 1
        
        new_state = self.update_state(self.state_history, action_input)
        
        print(new_state.shape, self.state.shape)
        # check fro increase
        if new_state[component_pos['LIT101']] > self.state[component_pos['LIT101']]:
            reward = new_state[component_pos['LIT101']] - self.state[component_pos['LIT101']]
        elif new_state[component_pos['LIT101']] == self.state[component_pos['LIT101']]:
            reward = -1
        elif new_state[component_pos['LIT101']] < self.state[component_pos['LIT101']]:
            reward = self.state[component_pos['LIT101']] - new_state[component_pos['LIT101']]
            
        print(f'lit:{self.state[component_pos['LIT101']], new_state[component_pos['LIT101']]} reward:{reward}')
        
        # update state
        self.state = new_state
    
        # reduce episode_length
        self.episode_length = self.episode_length - 1
        
            
        # Check if episode is done
        if self.episode_length <= 0:
            done = True
        else:
            done = False         
        
        return self.state, reward, done #, info

    def reset(self, model_input):
        # state history is 98 time steps preceding
        self.state_history = model_input
        # set start state as last row in modelinput
        self.state = self.state_history [0,-1,:]
        # reset episode
        self.episode_length = 60
        return self.state
        
    #def render(self):
        # for vis ie pygame
        #pass
    def render(self, screen, font):
        '''
        screen.fill((255, 255, 255))  # Fill the screen with white

        # Draw the temperature meter
        pygame.draw.rect(screen, (0, 0, 255), (100, 150, 50, 200))  # Blue background
        temp_height = (self.state - 0) / 100 * 200  # Scale temp to meter height
        pygame.draw.rect(screen, (255, 0, 0), (100, 350 - temp_height, 50, temp_height))  # Red temperature

        # Display temperature value
        temp_text = font.render(f'Temp: {self.state}°C', True, (0, 0, 0))
        screen.blit(temp_text, (200, 150))

        # Display shower length
        shower_text = font.render(f'Shower Time: {self.shower_length}s', True, (0, 0, 0))
        screen.blit(shower_text, (200, 200))
        '''
        pass

        



def main():
    # Initialize pygame
    pygame.init()
    screen = pygame.display.set_mode((600, 400))
    pygame.display.set_caption("SWAT Environment Visualization")

    # Font for rendering text
    font = pygame.font.SysFont('Arial', 24)

    # Create the environment
    env = SwatEnv()

    # Initialize game loop variables
    running = True
    clock = pygame.time.Clock()

    state = env.reset()  # Initialize environment state

    while running:
        # Handle events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Action logic (can be random for testing purposes)
        action = random.randint(0, 2)  # Replace with your agent's action

        # Perform step
        state, reward, done, info = env.step(action)

        # Render the environment
        env.render(screen, font)

        # Update the display
        pygame.display.flip()

        # End if the shower is done
        if done:
            state = env.reset()

        # Control frame rate (optional)
        clock.tick(30)  # 30 FPS

    pygame.quit()

if __name__ == "__main__":
    main()


In [63]:
reshaped_input[0,11,]

array([0.  , 0.98, 0.5 , 1.  , 0.92, 0.14, 0.54, 1.  , 1.  , 1.  , 0.  ,
       0.97, 0.94, 0.01, 0.5 , 1.  , 0.5 , 0.5 , 0.  , 1.  , 0.  , 0.79,
       0.5 , 0.19, 0.45, 0.8 , 0.65, 0.69, 0.53, 0.35, 0.44, 0.55, 0.34,
       0.24, 0.35, 0.  , 0.  ])

In [64]:
env = SwatEnv(reshaped_input)

init reached


/home/loz/Documents/GitHub/MSc-Project/venv/lib/python3.12/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [65]:
# sample from action space
#env.action_space.sample()

In [66]:
env.observation_space.sample()

array([[72.8331   , 53.644554 , 24.459045 , 69.5047   , 46.111897 ,
         2.356586 ,  1.9686879, 25.79574  , 11.025445 , 64.6745   ,
        74.21311  , 46.39377  , 92.801895 ,  4.132817 , 77.646515 ,
        56.19956  , 54.199818 ,  2.747938 , 70.82056  ,  6.1048083,
        74.884315 , 31.400608 , 66.17204  , 25.449844 ,  1.8726039,
        54.869495 , 52.435993 , 52.45739  , 57.20808  , 85.929245 ,
        40.183323 , 64.79173  , 41.286743 , 72.96217  , 87.14934  ,
        82.96504  ,  1.7474911]], dtype=float32)

In [67]:
# shown as high precision but actually 2 dp....
sampled_obs = env.observation_space.sample()
sampled_obs_rnd = np.round(sampled_obs, 2)
sampled_obs_rnd

array([[96.03, 51.49, 63.43, 66.96, 26.51, 60.73, 68.01, 75.68, 21.73,
        68.85, 46.2 , 78.06, 86.59, 17.11,  5.88,  0.25, 67.3 , 82.09,
        63.68, 13.85,  9.35, 82.79, 11.84, 81.93, 57.77,  2.31, 41.79,
        57.11, 46.37,  1.54, 38.73, 85.16, 95.68, 12.47, 62.99, 35.9 ,
        78.4 ]], dtype=float32)

In [68]:
print(env.new_state.shape, env.state.shape)

AttributeError: 'SwatEnv' object has no attribute 'new_state'

In [71]:
episodes = 10
for episode in range(1, episodes+1):
    env.reset(reshaped_input)
    state = env.state
    done = False
    score = 0 
    
    while not done:
        #env.render()
        # sample from 4 possible states 
        action = env.action_space.sample()
        # pass 1 x 37 array of SWat states
        #action = one_state
        #n_state, reward, done, info = env.step(action)
        n_state, reward, done = env.step(action)
        print(n_state)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
(37,) (37,)
lit:(0.84, 0.87357104) reward:0.033571038246154816
[ 0.07035083  0.87357104  0.07438044  0.32859194  0.6811092   0.07326411
  0.31588072  0.33774227  0.36133057  0.3562329   0.5816231   0.5626696
  0.43102434  0.36469772  0.1791623   0.64833075  0.09943427  0.06670187
 -0.19631171  0.49815795 -0.42096567  0.30042136  0.26147053  0.40290737
 -0.36592263  0.31323794  0.9475062   1.5687454   0.32463685  0.22808982
 -0.05658165 -0.2843256  -0.02670207 -0.45720348  0.00362511 -0.0113398
  0.00661582]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
(37,) (37,)
lit:(0.87357104, 0.87226355) reward:0.0013074874877929688
[ 0.13641639  0.87226355  0.09351127  0.51976323  0.77390563 -0.04399579
  0.3663203   0.55443203  0.5466116   0.568925    1.0055709   0.59039867
  0.5145221   0.4234682   0.14835584  0.5851477  -0.03659707 -0.02869247
 -0.01849513  0.49839392 -0.5027343   0.1233712   0.26020765  0.57161176
 -0.18110132  0.08525426  1.0580708   1.6501068  

# Create Deep Learning Model#

In [48]:
states = env.observation_space.shape
actions = env.action_space.n

In [33]:
states

(1,)

In [34]:
actions

3

In [35]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    #model.add(Dense(24, activation='relu', input_shape=(1,)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [36]:
#del model

In [37]:
model = build_model(states, actions)

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# Build Agent wit Keras-RL

In [39]:
#@tf.function(reduce_retracing=True)
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [40]:
dqn = build_agent(model, actions)

In [41]:
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

In [42]:


dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)


Training for 5000 steps ...
Interval 1 (0 steps performed)
 1423/10000 [===>..........................] - ETA: 1:36 - reward: -0.5264done, took 16.064 seconds


In [23]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 10.000, steps: 60
Episode 2: reward: 16.000, steps: 60
Episode 3: reward: 6.000, steps: 60
Episode 4: reward: 8.000, steps: 60
Episode 5: reward: 4.000, steps: 60
Episode 6: reward: 18.000, steps: 60
Episode 7: reward: -16.000, steps: 60
Episode 8: reward: -12.000, steps: 60
Episode 9: reward: -8.000, steps: 60
Episode 10: reward: -2.000, steps: 60
Episode 11: reward: -2.000, steps: 60
Episode 12: reward: 8.000, steps: 60
Episode 13: reward: -10.000, steps: 60
Episode 14: reward: 4.000, steps: 60
Episode 15: reward: 2.000, steps: 60
Episode 16: reward: 6.000, steps: 60
Episode 17: reward: -10.000, steps: 60
Episode 18: reward: 0.000, steps: 60
Episode 19: reward: 10.000, steps: 60
Episode 20: reward: -8.000, steps: 60
Episode 21: reward: -6.000, steps: 60
Episode 22: reward: 6.000, steps: 60
Episode 23: reward: 0.000, steps: 60
Episode 24: reward: -2.000, steps: 60
Episode 25: reward: 8.000, steps: 60
Episode 26: reward: 6.000, steps: 60


In [12]:
os.getcwd()

'/home/loz/Documents/GitHub/MSc-Project/final_code'

In [26]:
#os.listdir()

In [24]:
# get pre_trained model of SWaT
swat_lstm = tf.keras.models.load_model('swat_lstm_100s.keras')


OSError: Unable to synchronously open file (file signature not found)